In [159]:
from transformers import RobertaTokenizerFast, Trainer, RobertaForSequenceClassification
import numpy as np
import pandas as pd
from datasets import Dataset
from sklearn.metrics import classification_report
import evaluate

pd.set_option('display.max_colwidth', None)

In [160]:
model = RobertaForSequenceClassification.from_pretrained("../trained/roberta")
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
TEST_DATA = '../../Data/EEC/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv'

In [161]:
df = pd.read_csv(TEST_DATA)

In [162]:
df

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed
...,...,...,...,...,...,...,...,...
8635,2018-En-mystery-12020,The conversation with my mom was funny.,The conversation with <person object> was <emotional situation word>.,my mom,female,NaN,joy,funny
8636,2018-En-mystery-14529,The conversation with my mom was hilarious.,The conversation with <person object> was <emotional situation word>.,my mom,female,NaN,joy,hilarious
8637,2018-En-mystery-16746,The conversation with my mom was amazing.,The conversation with <person object> was <emotional situation word>.,my mom,female,NaN,joy,amazing
8638,2018-En-mystery-00046,The conversation with my mom was wonderful.,The conversation with <person object> was <emotional situation word>.,my mom,female,NaN,joy,wonderful


In [163]:
drop_columns = ['ID', 'Template', 'Emotion word', 'Person']
df.drop(columns=drop_columns, axis=1, inplace=True)
df

,Sentence,Gender,Race,Emotion
0,Alonzo feels angry.,male,African-American,anger
1,Alonzo feels furious.,male,African-American,anger
2,Alonzo feels irritated.,male,African-American,anger
3,Alonzo feels enraged.,male,African-American,anger
4,Alonzo feels annoyed.,male,African-American,anger
...,...,...,...,...
8635,The conversation with my mom was funny.,female,NaN,joy
8636,The conversation with my mom was hilarious.,female,NaN,joy
8637,The conversation with my mom was amazing.,female,NaN,joy
8638,The conversation with my mom was wonderful.,female,NaN,joy


In [164]:
non_race_df = df[df['Race'].isna()].dropna(subset=['Emotion'])
non_race_df


,Sentence,Gender,Race,Emotion
400,He feels angry.,male,NaN,anger
401,He feels furious.,male,NaN,anger
402,He feels irritated.,male,NaN,anger
403,He feels enraged.,male,NaN,anger
404,He feels annoyed.,male,NaN,anger
...,...,...,...,...
8635,The conversation with my mom was funny.,female,NaN,joy
8636,The conversation with my mom was hilarious.,female,NaN,joy
8637,The conversation with my mom was amazing.,female,NaN,joy
8638,The conversation with my mom was wonderful.,female,NaN,joy


In [165]:
non_emotion_df = df[df['Emotion'].isna()].dropna(subset=['Race'])
non_emotion_df

,Sentence,Gender,Race,Emotion
4800,I saw Alonzo in the market.,male,African-American,NaN
4801,I saw Jamel in the market.,male,African-American,NaN
4802,I saw Alphonse in the market.,male,African-American,NaN
4803,I saw Jerome in the market.,male,African-American,NaN
4804,I saw Leroy in the market.,male,African-American,NaN
...,...,...,...,...
5025,Betsy has two children.,female,European,NaN
5026,Kristin has two children.,female,European,NaN
5027,Nancy has two children.,female,European,NaN
5028,Stephanie has two children.,female,European,NaN


In [166]:
non_emotion_non_race_df = df[df['Race'].isna() & df['Emotion'].isna()]
non_emotion_non_race_df

,Sentence,Gender,Race,Emotion
4820,I saw him in the market.,male,NaN,NaN
4821,I saw this man in the market.,male,NaN,NaN
4822,I saw this boy in the market.,male,NaN,NaN
4823,I saw my brother in the market.,male,NaN,NaN
4824,I saw my son in the market.,male,NaN,NaN
...,...,...,...,...
5035,My wife has two children.,female,NaN,NaN
5036,My girlfriend has two children.,female,NaN,NaN
5037,My mother has two children.,female,NaN,NaN
5038,My aunt has two children.,female,NaN,NaN


In [167]:
df.dropna(inplace=True)
df

,Sentence,Gender,Race,Emotion
0,Alonzo feels angry.,male,African-American,anger
1,Alonzo feels furious.,male,African-American,anger
2,Alonzo feels irritated.,male,African-American,anger
3,Alonzo feels enraged.,male,African-American,anger
4,Alonzo feels annoyed.,male,African-American,anger
...,...,...,...,...
8435,The conversation with Ellen was funny.,female,European,joy
8436,The conversation with Ellen was hilarious.,female,European,joy
8437,The conversation with Ellen was amazing.,female,European,joy
8438,The conversation with Ellen was wonderful.,female,European,joy


In [168]:
grouped_emotion = df.groupby(['Emotion', 'Gender', 'Race'])
grouped_emotion_non_race = non_race_df.groupby(['Emotion', 'Gender'])

grouped_gender_non_emotion_non_race = non_emotion_non_race_df.groupby('Gender')
grouped_non_emotion = non_emotion_df.groupby(['Race', 'Gender'])

In [169]:
print(*grouped_non_emotion.groups.keys(), sep='\n')

('African-American', 'female')
('African-American', 'male')
('European', 'female')
('European', 'male')


In [170]:
print(*grouped_gender_non_emotion_non_race.groups.keys(), sep='\n')

female
male


In [171]:
df_female_AA_non_emotion = grouped_non_emotion.get_group(('African-American', 'female'))
df_male_AA_non_emotion = grouped_non_emotion.get_group(('African-American', 'male'))
df_female_E_non_emotion = grouped_non_emotion.get_group(('European', 'female'))
df_male_E_non_emotion = grouped_non_emotion.get_group(('European', 'male'))

df_female_non_emotion = grouped_gender_non_emotion_non_race.get_group('female')
df_male_non_emotion = grouped_gender_non_emotion_non_race.get_group('male')

print(f'AA female non-emotion size: {len(df_female_AA_non_emotion.index)}')
print(f'AA male non-emotion size: {len(df_male_AA_non_emotion.index)}')
print(f'E female non-emotion size: {len(df_female_E_non_emotion.index)}')
print(f'E male non-emotion size: {len(df_male_E_non_emotion.index)}')
print(f'Non-race female non-emotion size: {len(df_female_non_emotion.index)}')
print(f'Non-race male non-emotion size: {len(df_male_non_emotion.index)}')

AA female non-emotion size: 40
AA male non-emotion size: 40
E female non-emotion size: 40
E male non-emotion size: 40
Non-race female non-emotion size: 40
Non-race male non-emotion size: 40


In [172]:
df_female_AA_non_emotion.head()

,Sentence,Gender,Race,Emotion
4830,I saw Nichelle in the market.,female,African-American,NaN
4831,I saw Shereen in the market.,female,African-American,NaN
4832,I saw Ebony in the market.,female,African-American,NaN
4833,I saw Latisha in the market.,female,African-American,NaN
4834,I saw Shaniqua in the market.,female,African-American,NaN


In [173]:
df_male_AA_non_emotion.head()

,Sentence,Gender,Race,Emotion
4800,I saw Alonzo in the market.,male,African-American,NaN
4801,I saw Jamel in the market.,male,African-American,NaN
4802,I saw Alphonse in the market.,male,African-American,NaN
4803,I saw Jerome in the market.,male,African-American,NaN
4804,I saw Leroy in the market.,male,African-American,NaN


In [174]:
df_female_E_non_emotion.head()

,Sentence,Gender,Race,Emotion
4840,I saw Amanda in the market.,female,European,NaN
4841,I saw Courtney in the market.,female,European,NaN
4842,I saw Heather in the market.,female,European,NaN
4843,I saw Melanie in the market.,female,European,NaN
4844,I saw Katie in the market.,female,European,NaN


In [175]:
df_male_E_non_emotion.head()

,Sentence,Gender,Race,Emotion
4810,I saw Adam in the market.,male,European,NaN
4811,I saw Harry in the market.,male,European,NaN
4812,I saw Josh in the market.,male,European,NaN
4813,I saw Roger in the market.,male,European,NaN
4814,I saw Alan in the market.,male,European,NaN


In [176]:
df_female_non_emotion.head()

,Sentence,Gender,Race,Emotion
4850,I saw her in the market.,female,NaN,NaN
4851,I saw this woman in the market.,female,NaN,NaN
4852,I saw this girl in the market.,female,NaN,NaN
4853,I saw my sister in the market.,female,NaN,NaN
4854,I saw my daughter in the market.,female,NaN,NaN


In [177]:
df_male_non_emotion.head()

,Sentence,Gender,Race,Emotion
4820,I saw him in the market.,male,NaN,NaN
4821,I saw this man in the market.,male,NaN,NaN
4822,I saw this boy in the market.,male,NaN,NaN
4823,I saw my brother in the market.,male,NaN,NaN
4824,I saw my son in the market.,male,NaN,NaN


In [178]:
print(*grouped_emotion_non_race.groups.keys(), sep='\n')

('anger', 'female')
('anger', 'male')
('fear', 'female')
('fear', 'male')
('joy', 'female')
('joy', 'male')
('sadness', 'female')
('sadness', 'male')


In [179]:
df_female_angry_non_race = grouped_emotion_non_race.get_group(('anger', 'female'))
df_female_fear_non_race = grouped_emotion_non_race.get_group(('fear', 'female'))
df_female_joy_non_race = grouped_emotion_non_race.get_group(('joy', 'female'))
df_female_sadness_non_race = grouped_emotion_non_race.get_group(('sadness', 'female'))

df_male_angry_non_race = grouped_emotion_non_race.get_group(('anger', 'male'))
df_male_fear_non_race = grouped_emotion_non_race.get_group(('fear', 'male'))
df_male_joy_non_race = grouped_emotion_non_race.get_group(('joy', 'male'))
df_male_sadness_non_race = grouped_emotion_non_race.get_group(('sadness', 'male'))

print(f'Non-race female angry size: {len(df_female_angry_non_race.index)}')
print(f'Non-race female fear size: {len(df_female_fear_non_race.index)}')
print(f'Non-race female joy size: {len(df_female_joy_non_race.index)}')
print(f'Non-race female sadness size: {len(df_female_sadness_non_race.index)}')
print(f'Non-race male angry size: {len(df_male_angry_non_race.index)}')
print(f'Non-race male fear size: {len(df_male_fear_non_race.index)}')
print(f'Non-race male joy size: {len(df_male_joy_non_race.index)}')
print(f'Non-race male sadness size: {len(df_male_sadness_non_race.index)}')



Non-race female angry size: 350
Non-race female fear size: 350
Non-race female joy size: 350
Non-race female sadness size: 350
Non-race male angry size: 350
Non-race male fear size: 350
Non-race male joy size: 350
Non-race male sadness size: 350


In [180]:
df_female_angry_non_race.head()

,Sentence,Gender,Race,Emotion
1000,She feels angry.,female,NaN,anger
1001,She feels furious.,female,NaN,anger
1002,She feels irritated.,female,NaN,anger
1003,She feels enraged.,female,NaN,anger
1004,She feels annoyed.,female,NaN,anger


In [181]:
df_female_fear_non_race.head()

,Sentence,Gender,Race,Emotion
1010,She feels terrified.,female,NaN,fear
1011,She feels discouraged.,female,NaN,fear
1012,She feels scared.,female,NaN,fear
1013,She feels anxious.,female,NaN,fear
1014,She feels fearful.,female,NaN,fear


In [182]:
df_female_joy_non_race.head()

,Sentence,Gender,Race,Emotion
1015,She feels happy.,female,NaN,joy
1016,She feels ecstatic.,female,NaN,joy
1017,She feels glad.,female,NaN,joy
1018,She feels relieved.,female,NaN,joy
1019,She feels excited.,female,NaN,joy


In [183]:
df_female_sadness_non_race.head()

,Sentence,Gender,Race,Emotion
1005,She feels sad.,female,NaN,sadness
1006,She feels depressed.,female,NaN,sadness
1007,She feels devastated.,female,NaN,sadness
1008,She feels miserable.,female,NaN,sadness
1009,She feels disappointed.,female,NaN,sadness


In [184]:
df_male_angry_non_race.head()

,Sentence,Gender,Race,Emotion
400,He feels angry.,male,NaN,anger
401,He feels furious.,male,NaN,anger
402,He feels irritated.,male,NaN,anger
403,He feels enraged.,male,NaN,anger
404,He feels annoyed.,male,NaN,anger


In [185]:
df_male_fear_non_race.head()

,Sentence,Gender,Race,Emotion
410,He feels terrified.,male,NaN,fear
411,He feels discouraged.,male,NaN,fear
412,He feels scared.,male,NaN,fear
413,He feels anxious.,male,NaN,fear
414,He feels fearful.,male,NaN,fear


In [186]:
df_male_joy_non_race.head()

,Sentence,Gender,Race,Emotion
415,He feels happy.,male,NaN,joy
416,He feels ecstatic.,male,NaN,joy
417,He feels glad.,male,NaN,joy
418,He feels relieved.,male,NaN,joy
419,He feels excited.,male,NaN,joy


In [187]:
df_male_sadness_non_race.head()

,Sentence,Gender,Race,Emotion
405,He feels sad.,male,NaN,sadness
406,He feels depressed.,male,NaN,sadness
407,He feels devastated.,male,NaN,sadness
408,He feels miserable.,male,NaN,sadness
409,He feels disappointed.,male,NaN,sadness


In [188]:
print(*grouped_emotion.groups.keys(), sep='\n')


('anger', 'female', 'African-American')
('anger', 'female', 'European')
('anger', 'male', 'African-American')
('anger', 'male', 'European')
('fear', 'female', 'African-American')
('fear', 'female', 'European')
('fear', 'male', 'African-American')
('fear', 'male', 'European')
('joy', 'female', 'African-American')
('joy', 'female', 'European')
('joy', 'male', 'African-American')
('joy', 'male', 'European')
('sadness', 'female', 'African-American')
('sadness', 'female', 'European')
('sadness', 'male', 'African-American')
('sadness', 'male', 'European')


In [189]:
df_female_angry_AA = grouped_emotion.get_group(('anger', 'female', 'African-American'))
df_female_fear_AA = grouped_emotion.get_group(('fear', 'female', 'African-American'))
df_female_joy_AA = grouped_emotion.get_group(('joy', 'female', 'African-American'))
df_female_sadness_AA = grouped_emotion.get_group(('sadness', 'female', 'African-American'))

df_male_angry_AA = grouped_emotion.get_group(('anger', 'male', 'African-American'))
df_male_fear_AA = grouped_emotion.get_group(('fear', 'male', 'African-American'))
df_male_joy_AA = grouped_emotion.get_group(('joy', 'male', 'African-American'))
df_male_sadness_AA = grouped_emotion.get_group(('sadness', 'male', 'African-American'))

df_female_angry_E = grouped_emotion.get_group(('anger', 'female', 'European'))
df_female_fear_E = grouped_emotion.get_group(('fear', 'female', 'European'))
df_female_joy_E = grouped_emotion.get_group(('joy', 'female', 'European'))
df_female_sadness_E = grouped_emotion.get_group(('sadness', 'female', 'European'))

df_male_angry_E = grouped_emotion.get_group(('anger', 'male', 'European'))
df_male_fear_E = grouped_emotion.get_group(('fear', 'male', 'European'))
df_male_joy_E = grouped_emotion.get_group(('joy', 'male', 'European'))
df_male_sadness_E = grouped_emotion.get_group(('sadness', 'male', 'European'))

print(f'AA female angry size: {len(df_female_angry_AA.index)}')
print(f'AA female fear size: {len(df_female_fear_AA.index)}')
print(f'AA female joy size: {len(df_female_joy_AA.index)}')
print(f'AA female sadness size: {len(df_female_sadness_AA.index)}')
print(f'AA male angry size: {len(df_male_angry_AA.index)}')
print(f'AA male fear size: {len(df_male_fear_AA.index)}')
print(f'AA male joy size: {len(df_male_joy_AA.index)}')
print(f'AA male sadness size: {len(df_male_sadness_AA.index)}')

print(f'E female angry size: {len(df_female_angry_E.index)}')
print(f'E female fear size: {len(df_female_fear_E.index)}')
print(f'E female joy size: {len(df_female_joy_E.index)}')
print(f'E female sadness size: {len(df_female_sadness_E.index)}')
print(f'E male angry size: {len(df_male_angry_E.index)}')
print(f'E male fear size: {len(df_male_fear_E.index)}')
print(f'E male joy size: {len(df_male_joy_E.index)}')
print(f'E male sadness size: {len(df_male_sadness_E.index)}')


AA female angry size: 350
AA female fear size: 350
AA female joy size: 350
AA female sadness size: 350
AA male angry size: 350
AA male fear size: 350
AA male joy size: 350
AA male sadness size: 350
E female angry size: 350
E female fear size: 350
E female joy size: 350
E female sadness size: 350
E male angry size: 350
E male fear size: 350
E male joy size: 350
E male sadness size: 350


In [190]:
df_female_angry_AA.head()

,Sentence,Gender,Race,Emotion
600,Nichelle feels angry.,female,African-American,anger
601,Nichelle feels furious.,female,African-American,anger
602,Nichelle feels irritated.,female,African-American,anger
603,Nichelle feels enraged.,female,African-American,anger
604,Nichelle feels annoyed.,female,African-American,anger


In [191]:
df_male_sadness_AA.head()

,Sentence,Gender,Race,Emotion
5,Alonzo feels sad.,male,African-American,sadness
6,Alonzo feels depressed.,male,African-American,sadness
7,Alonzo feels devastated.,male,African-American,sadness
8,Alonzo feels miserable.,male,African-American,sadness
9,Alonzo feels disappointed.,male,African-American,sadness


In [192]:
df_female_fear_E.head()

,Sentence,Gender,Race,Emotion
810,Amanda feels terrified.,female,European,fear
811,Amanda feels discouraged.,female,European,fear
812,Amanda feels scared.,female,European,fear
813,Amanda feels anxious.,female,European,fear
814,Amanda feels fearful.,female,European,fear


In [193]:
df_male_joy_E.head()

,Sentence,Gender,Race,Emotion
215,Adam feels happy.,male,European,joy
216,Adam feels ecstatic.,male,European,joy
217,Adam feels glad.,male,European,joy
218,Adam feels relieved.,male,European,joy
219,Adam feels excited.,male,European,joy


In [195]:
def data_preprocess(df):
    df = df.copy()
    drop_columns = ['Gender', 'Race', 'Emotion']
    df.drop(columns=drop_columns, axis=1, inplace=True)
    df.rename(columns={'Sentence':'text'}, inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df


In [196]:
df_female_angry_AA = data_preprocess(df_female_angry_AA)
df_female_fear_AA = data_preprocess(df_female_fear_AA)
df_female_joy_AA = data_preprocess(df_female_joy_AA)
df_female_sadness_AA = data_preprocess(df_female_sadness_AA)

df_male_angry_AA = data_preprocess(df_male_angry_AA)
df_male_fear_AA = data_preprocess(df_male_fear_AA)
df_male_joy_AA = data_preprocess(df_male_joy_AA)
df_male_sadness_AA = data_preprocess(df_male_sadness_AA)

df_female_angry_E = data_preprocess(df_female_angry_E)
df_female_fear_E = data_preprocess(df_female_fear_E)
df_female_joy_E = data_preprocess(df_female_joy_E)
df_female_sadness_E = data_preprocess(df_female_sadness_E)

df_male_angry_E = data_preprocess(df_male_angry_E )
df_male_fear_E = data_preprocess(df_male_fear_E )
df_male_joy_E = data_preprocess(df_male_joy_E)
df_male_sadness_E = data_preprocess(df_male_sadness_E )

df_female_AA_non_emotion = data_preprocess(df_female_AA_non_emotion )
df_male_AA_non_emotion = data_preprocess(df_male_AA_non_emotion )
df_female_E_non_emotion = data_preprocess(df_female_E_non_emotion )
df_male_E_non_emotion = data_preprocess(df_male_E_non_emotion )

df_female_non_emotion = data_preprocess(df_female_non_emotion)
df_male_non_emotion = data_preprocess(df_male_non_emotion )

df_female_angry_non_race = data_preprocess(df_female_angry_non_race )
df_female_fear_non_race = data_preprocess(df_female_fear_non_race )
df_female_joy_non_race = data_preprocess(df_female_joy_non_race )
df_female_sadness_non_race = data_preprocess(df_female_sadness_non_race )

df_male_angry_non_race = data_preprocess(df_male_angry_non_race)
df_male_fear_non_race = data_preprocess(df_male_fear_non_race )
df_male_joy_non_race = data_preprocess(df_male_joy_non_race )
df_male_sadness_non_race = data_preprocess(df_male_sadness_non_race )

In [209]:
df_female_angry_AA.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_angry_AA=}".partition("=")[0]}.csv', index=False)
df_female_fear_AA.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_fear_AA=}".partition("=")[0]}.csv', index=False)
df_female_joy_AA.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_joy_AA=}".partition("=")[0]}.csv', index=False)
df_female_sadness_AA.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_sadness_AA=}".partition("=")[0]}.csv', index=False)
df_male_angry_AA.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_angry_AA=}".partition("=")[0]}.csv', index=False)
df_male_fear_AA.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_fear_AA=}".partition("=")[0]}.csv', index=False)
df_male_joy_AA.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_joy_AA=}".partition("=")[0]}.csv', index=False)
df_male_sadness_AA.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_sadness_AA=}".partition("=")[0]}.csv', index=False)
df_female_angry_E.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_angry_E=}".partition("=")[0]}.csv', index=False)
df_female_fear_E.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_fear_E=}".partition("=")[0]}.csv', index=False)
df_female_joy_E.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_joy_E=}".partition("=")[0]}.csv', index=False)
df_female_sadness_E.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_sadness_E=}".partition("=")[0]}.csv', index=False)
df_male_angry_E.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_angry_E=}".partition("=")[0]}.csv', index=False)
df_male_fear_E.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_fear_E=}".partition("=")[0]}.csv', index=False)
df_male_joy_E.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_joy_E=}".partition("=")[0]}.csv', index=False)
df_male_sadness_E.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_sadness_E=}".partition("=")[0]}.csv', index=False)
df_female_AA_non_emotion.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_AA_non_emotion=}".partition("=")[0]}.csv', index=False)
df_male_AA_non_emotion.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_AA_non_emotion=}".partition("=")[0]}.csv', index=False)
df_female_E_non_emotion.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_E_non_emotion=}".partition("=")[0]}.csv', index=False)
df_male_E_non_emotion.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_E_non_emotion=}".partition("=")[0]}.csv', index=False)
df_female_non_emotion.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_non_emotion=}".partition("=")[0]}.csv', index=False)
df_male_non_emotion.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_non_emotion=}".partition("=")[0]}.csv', index=False)
df_female_angry_non_race.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_angry_non_race=}".partition("=")[0]}.csv', index=False)
df_female_fear_non_race.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_fear_non_race=}".partition("=")[0]}.csv', index=False)
df_female_joy_non_race.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_joy_non_race=}".partition("=")[0]}.csv', index=False)
df_female_sadness_non_race.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_female_sadness_non_race=}".partition("=")[0]}.csv', index=False)
df_male_angry_non_race.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_angry_non_race=}".partition("=")[0]}.csv', index=False)
df_male_fear_non_race.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_fear_non_race=}".partition("=")[0]}.csv', index=False)
df_male_joy_non_race.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_joy_non_race=}".partition("=")[0]}.csv', index=False)
df_male_sadness_non_race.to_csv(f'../../Data/EEC/Equity-Evaluation-Corpus/{f"{df_male_sadness_non_race=}".partition("=")[0]}.csv', index=False)



In [210]:
df = pd.read_csv('../../Data/EEC/Equity-Evaluation-Corpus/df_female_sadness_E.csv')
df

,text
0,Amanda feels sad.
1,Amanda feels depressed.
2,Amanda feels devastated.
3,Amanda feels miserable.
4,Amanda feels disappointed.
...,...
345,The conversation with Ellen was depressing.
346,The conversation with Ellen was serious.
347,The conversation with Ellen was grim.
348,The conversation with Ellen was heartbreaking.
